In [1]:
from train import train, class_acc
from model import Model
from dataset import TrainDataset, TestDataset
from test import predictTestData

In [2]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from numpy import asarray
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms as T
from torchvision.transforms import Resize, ToTensor, Normalize

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torchvision.models as models
from torchsummary import summary

from tqdm import tqdm

from GPUtil import showUtilization as gpu_usage
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(device))

Tesla V100-PCIE-32GB


In [4]:
age_dir = '/opt/ml/split_labels/age/'
train = pd.read_csv(age_dir + 'train.csv')
valid = pd.read_csv(age_dir + 'valid.csv')

In [5]:
test = pd.read_csv('/opt/ml/input/data/eval/info.csv')

In [6]:
AgeModel = Model(num_classes=3).to(device)

In [7]:
X_train, y_train = train['path'], train['age']
X_valid, y_valid = valid['path'], valid['age']
# X_train.head()

In [8]:
test['path'] = [os.path.join('/opt/ml/input/data/eval/images/', img) for img in test.ImageID]

In [9]:
CROP_SIZE = 384
RESIZE = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

cutout = A.Compose([
    A.OpticalDistortion(p=0.5),
    A.OneOf([
        A.Cutout(p=1, num_holes=10, max_h_size=10, max_w_size=6),
        A.GridDistortion(p=1, num_steps=5, interpolation=1, border_mode=1, value=(0, 0, 0)),
    ], p=0.5),
    A.CenterCrop(height=CROP_SIZE+80, width=CROP_SIZE),
    A.Resize(height=RESIZE, width=RESIZE),
    A.Normalize(mean=mean, std=std),
    ToTensorV2(),
])

transform = A.Compose([
    A.CenterCrop(height=CROP_SIZE+80, width=CROP_SIZE),
    A.Resize(height=RESIZE, width=RESIZE),
    A.Normalize(mean=mean, std=std),
    ToTensorV2(),
])

/opt/conda/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:688: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(


In [10]:
train_dataset = TrainDataset(X_train, y_train, cutout)
valid_dataset = TrainDataset(X_valid, y_valid, transform)

100%|██████████| 1890/1890 [00:06<00:00, 277.46it/s]


In [11]:
test_dataset = TestDataset(test['path'], transform)

100%|██████████| 12600/12600 [00:44<00:00, 283.68it/s]


In [ ]:
BATCH_SIZE = 64
_train_loader = DataLoader(_train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
_valid_loader = DataLoader(_valid_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

In [ ]:
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR

LEARNING_RATE = 1e-4
criterion = nn.CrossEntropyLoss()
optG = Adam(GenderModel.parameters(), lr=LEARNING_RATE)
schG = StepLR(optG, step_size=5, gamma=0.5)

In [ ]:
train(
    model = AgeModel,
    optimizer = optG,
    criterion = criterion,
    train_loader = gender_train_loader,
    val_loader = gender_valid_loader,
    device = device,
    scheduler = schG,
    writer = writer,
    epochs = 3
)